In [1]:
#pip install -q super-gradients==3.1.1
#!pip install -q roboflow
#!pip install -q supervision

In [2]:
#pip install -q super-gradients==3.1.1

In [3]:
import os
HOME = os.getcwd()
print(HOME)


/Users/mautushid/github/AI_project


In [4]:
import torch

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'


In [17]:
DEVICE

'cpu'

In [5]:
from super_gradients.training import models

model = models.get(MODEL_ARCH, pretrained_weights="coco").to(DEVICE)


[2023-10-23 14:23:50] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into /Users/mautushid/sg_logs/console.log


[2023-10-23 14:23:51] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-10-23 14:23:52] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-10-23 14:23:52] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-10-23 14:23:52] WARNING - export.py - Failed to import pytorch_quantization
[2023-10-23 14:23:52] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-10-23 14:23:52] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.
[2023-10-23 14:23:52] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


In [6]:
#####Finetunning#######



In [7]:
from super_gradients.training import Trainer

CHECKPOINT_DIR = 'checkpoints'
trainer = Trainer(experiment_name='yolonas_AI',
                  ckpt_root_dir=CHECKPOINT_DIR)


In [8]:
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val


In [9]:
dataset_params = {
    'data_dir': 'data',
    'train_images_dir': 'train/train_img',
    'train_labels_dir': 'train/trainyolo',
    'val_images_dir': 'val/val_img',
    'val_labels_dir': 'val/valyolo',
    'test_images_dir': 'test/test_img',
    'test_labels_dir': 'test/testyolo',
    'classes': ['Pedestrian', 'Cyclist', 'Car', 'Truck', 'Tram',  'Tricycle'],
}


In [10]:
#Class = dataset_params['classes']

In [11]:
#len(classess)


In [12]:
from IPython.display import clear_output

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 16,
        'num_workers': 2
    }
)

clear_output()


In [13]:
train_data.dataset.transforms


[DetectionMosaic('additional_samples_count': 3, 'non_empty_targets': False, 'prob': 1.0, 'input_dim': [640, 640], 'enable_mosaic': True, 'border_value': 114),
 DetectionRandomAffine('additional_samples_count': 0, 'non_empty_targets': False, 'degrees': 10.0, 'translate': 0.1, 'scale': [0.1, 2], 'shear': 2.0, 'target_size': [640, 640], 'enable': True, 'filter_box_candidates': True, 'wh_thr': 2, 'ar_thr': 20, 'area_thr': 0.1, 'border_value': 114),
 DetectionMixup('additional_samples_count': 1, 'non_empty_targets': True, 'input_dim': [640, 640], 'mixup_scale': [0.5, 1.5], 'prob': 1.0, 'enable_mixup': True, 'flip_prob': 0.5, 'border_value': 114),
 DetectionHSV('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 1.0, 'hgain': 5, 'sgain': 30, 'vgain': 30, 'bgr_channels': (0, 1, 2), '_additional_channels_warned': False),
 DetectionHorizontalFlip('additional_samples_count': 0, 'non_empty_targets': False, 'prob': 0.5, 'max_targets': 120),
 DetectionPaddedRescale('swap': (2, 0, 1),

In [14]:
#calling the pretrained model
from super_gradients.training import models
model = models.get('yolo_nas_l',
                   num_classes= len(dataset_params['classes']),
                   pretrained_weights="coco"
                   )


Caching annotations: 100%|██████████| 20/20 [00:00<00:00, 328.36it/s]
[2023-10-23 14:23:53] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


In [15]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models": True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": 5,
    "mixed_precision": False,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls = len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}


In [ ]:
trainer.train(model=model,
              training_params=train_params,
              train_loader=train_data,
              valid_loader=val_data)


[2023-10-23 14:23:54] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}


The console stream is now moved to checkpoints/yolonas_AI/console_Oct23_14_24_06.txt


[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[2023-10-23 14:24:22] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               0          (0 available on the machine)
    - Dataset size:                 49         (len(train_set))
    - Batch size per GPU:           16         (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             16         (num_gpus * batch_size)
    - Effective Batch size:         16         (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         3          (len(train_lo

SUMMARY OF EPOCH 0
├── Training
│   ├── Ppyoloeloss/loss = 3.5902
│   ├── Ppyoloeloss/loss_cls = 2.2648
│   ├── Ppyoloeloss/loss_dfl = 1.1352
│   └── Ppyoloeloss/loss_iou = 0.3031
└── Validation
    ├── F1@0.50 = 0.0
    ├── Map@0.50 = 0.0113
    ├── Ppyoloeloss/loss = 3.5584
    ├── Ppyoloeloss/loss_cls = 2.2829
    ├── Ppyoloeloss/loss_dfl = 1.1302
    ├── Ppyoloeloss/loss_iou = 0.2842
    ├── Precision@0.50 = 0.0
    └── Recall@0.50 = 0.0



[2023-10-23 14:27:08] INFO - base_sg_logger.py - Checkpoint saved in checkpoints/yolonas_AI/ckpt_best.pth
[2023-10-23 14:27:08] INFO - sg_trainer.py - Best checkpoint overriden: validation mAP@0.50: 0.01133600901812315
Train epoch 1:   0%|          | 0/3 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Validation epoch 1:   0%|          | 0/2 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after pa

...

Train epoch 4:   0%|          | 0/3 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Validation epoch 4:   0%|          | 0/2 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Validation epoch 4: 100%|██████████| 2/2 [00:33<00:00, 16.91s/it]


SUMMARY OF EPOCH 4
├── Training
│   ├── Ppyoloeloss/loss = 2.3917
│   │   ├── Best until now = 2.5092 (↘ -0.1175)
│   │   └── Epoch N-1      = 2.5092 (↘ -0.1175)
│   ├── Ppyoloeloss/loss_cls = 1.139
│   │   ├── Best until now = 1.2145 (↘ -0.0755)
│   │   └── Epoch N-1      = 1.2145 (↘ -0.0755)
│   ├── Ppyoloeloss/loss_dfl = 1.0327
│   │   ├── Best until now = 1.0139 (↗ 0.0188)
│   │   └── Epoch N-1      = 1.0655 (↘ -0.0328)
│   └── Ppyoloeloss/loss_iou = 0.2946
│       ├── Best until now = 0.2813 (↗ 0.0133)
│       └── Epoch N-1      = 0.3048 (↘ -0.0102)
└── Validation
    ├── F1@0.50 = 0.0093
    │   ├── Best until now = 0.0476 (↘ -0.0383)
    │   └── Epoch N-1      = 0.013  (↘ -0.0037)
    ├── Map@0.50 = 0.0087
    │   ├── Best until now = 0.0704 (↘ -0.0617)
    │   └── Epoch N-1      = 0.0078 (↗ 0.0009)
    ├── Ppyoloeloss/loss = 9.0079
    │   ├── Best until now = 2.4848 (↗ 6.5231)
    │   └── Epoch N-1      = 5.9003 (↗ 3.1076)
    ├── Ppyoloeloss/loss_cls = 6.6674
    │   ├── Best

[2023-10-23 14:38:38] INFO - sg_trainer.py - RUNNING ADDITIONAL TEST ON THE AVERAGED MODEL...
Validation epoch 5:   0%|          | 0/2 [00:00<?, ?it/s][W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
Validation epoch 5: 100%|██████████| 2/2 [00:21<00:00,  9.04s/it]

Validation epoch 5: 100%|██████████| 2/2 [00:31<00:00, 15.63s/it]
[2023-10-23 14:39:09] INFO - base_sg_logger.py - [CLEANUP] - Successfully stopped system monitoring process


SUMMARY OF EPOCH 5
├── Training
│   ├── Ppyoloeloss/loss = 2.3917
│   │   ├── Best until now = 2.5092 (↘ -0.1175)
│   │   └── Epoch N-1      = 2.5092 (↘ -0.1175)
│   ├── Ppyoloeloss/loss_cls = 1.139
│   │   ├── Best until now = 1.2145 (↘ -0.0755)
│   │   └── Epoch N-1      = 1.2145 (↘ -0.0755)
│   ├── Ppyoloeloss/loss_dfl = 1.0327
│   │   ├── Best until now = 1.0139 (↗ 0.0188)
│   │   └── Epoch N-1      = 1.0655 (↘ -0.0328)
│   └── Ppyoloeloss/loss_iou = 0.2946
│       ├── Best until now = 0.2813 (↗ 0.0133)
│       └── Epoch N-1      = 0.3048 (↘ -0.0102)
└── Validation
    ├── F1@0.50 = 0.0375
    │   ├── Best until now = 0.0476 (↘ -0.0101)
    │   └── Epoch N-1      = 0.0093 (↗ 0.0282)
    ├── Map@0.50 = 0.0847
    │   ├── Best until now = 0.0704 (↗ 0.0143)
    │   └── Epoch N-1      = 0.0087 (↗ 0.076)
    ├── Ppyoloeloss/loss = 2.9932
    │   ├── Best until now = 2.4848 (↗ 0.5084)
    │   └── Epoch N-1      = 9.0079 (↘ -6.0147)
    ├── Ppyoloeloss/loss_cls = 1.8852
    │   ├── Best u